# Importing dependencies

In [1]:
from deepface import DeepFace
import pandas as pd
import cv2
import numpy as np
import os
import matplotlib as plt
import math
import glob
import time 
#I've added timers at the most computationally expensive places so that you can gauge how much detail is feasible for you to achieve depending on your hardware

# Splitting the relevant videos into frames

**In here, you can set:**
* the frames/second rate
* the input folder
* the output folder
* the naming system you want for your output

In [5]:
#For some reason he doesn't like backslashes when specifying file paths so make sure you use frontslashes and put an 'r' infront of the path
#Also doesn't like it if your path is not on your local machine (e.g. a OneDrive file)
#PAY ATTENTION TO THE SLASHES in the file path naming

# #This line starts a timer
# start = time.time()

pathOut = r'C:/Users/chazzers/Desktop/Recordings/Frames/'
count = 0
counter = 1
listing = os.listdir(r'C:/Users/chazzers/Desktop/Recordings/Charles')
for vid in listing:
    vid = r'C:/Users/chazzers/Desktop/Recordings/Charles/' + vid
    cap = cv2.VideoCapture(vid)
    count = 0
    counter += 1
    success = True
    while success:
        success,image = cap.read()
        print('read a new frame:',success)
        if count%7 == 0 :
              cv2.imwrite(pathOut + 'video%d'%counter + 'frame%d.jpg'%count ,image)
        count+=1

# #This line stops the timer
# elapsed_time_split = (time.time() - start) 

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new 

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new 

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new 

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new 

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new 

...

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new 

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new 

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new 

read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: False


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-2b5g8ysb\opencv\modules\imgcodecs\src\loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [22]:
#Dividing the total elapsed time by the number images split to get the time to extract one frame from the video
# splitsec = elapsed_time_split/count
# print(splitsec)

* Use the code below to split videos into *ALL* its frames (bye bye CPU)

In [23]:
# #splitting the video into individual frames

# # Playing video from file:
# cap = cv2.VideoCapture(test_video)

# try:
#     if not os.path.exists('video_frames'):
#         os.makedirs('video_frames')
# except OSError:
#     print ('Error: Creating directory of data')

# currentFrame = 0
# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     # Saves image of the current frame in jpg file
#     name = './test_videodata/frame' + str(currentFrame) + '.jpg'
#     print ('Creating...' + name)
#     cv2.imwrite(name, frame)

#     # To stop duplicate images
#     currentFrame += 1

# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

**There is a bit of a weird tradeoff here:**
* Either You split the video into ALL its frames and it takes forever to split the video and you run out of foccen storage memory
* OR You split the video every *n* frames (say 30- that's 1 img/second) and you analyze all of them but you dont get as granular a feedback

---

# Analyzing frames

* Change the filename to include the name of the relevant video below
* i.e. r'C:/Users/chazzers/Desktop/Recordings/Frames/video2.jpg' if you want video 2 
* or r'C:/Users/chazzers/Desktop/Recordings/Frames/video3.jpg' if you want video 3 etc.
* or r'C:/Users/chazzers/Desktop/Recordings/Frames/.jpg' if you want all the videos in one go.

In [4]:
# #putting image frames into an array so that DeepFace can run on them

# # #This line starts a timer
# # start = time.time()


# img_array2 = []
# for filename in glob.glob(r'C:/Users/chazzers/Desktop/Recordings/Frames/video2*.jpg'):
#     img = cv2.imread(filename)
#     height, width, layers = img.shape
#     size = (width,height)
#     img_array2.append(img)
    
# # #This line stops the timer    
# # elapsed_time_fl = (time.time() - start) 

In [6]:
img_array2 = []
for filename in glob.glob(r'C:/Users/chazzers/Desktop/Recordings/Frames/video2*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array2.append(img)

In [ ]:
# The cell below tells you how long it is taking to add one image to the array we are going to analyze
# On this example it is taking 0.02 sec

In [4]:
# #Dividing the total elapsed time by the number images appended to the array to get the time to extract append one image to the array
# arraysec = elapsed_time_fl/len(img_array)
# print(arraysec)

In [7]:
#Running deepface on the array just created
df_test2 = DeepFace.analyze(img_path=img_array2, actions = ['emotion'],enforce_detection=False)
# Alos possible to do 'age', 'gender', 'race' but increases the amount of time taken massively

Analyzing: 100%|███████████████████████████████████████████████████████████████████| 1863/1863 [09:31<00:00,  3.26it/s]


In [8]:
# This returns a python dictionary of emotions
df_test2

{'instance_1': {'emotion': {'angry': 2.253176592479741,
   'disgust': 3.0694569783553856e-05,
   'fear': 47.920957083759134,
   'happy': 8.417915116002846e-05,
   'sad': 43.06679922740455,
   'surprise': 0.00012600553032634566,
   'neutral': 6.7588325893183105},
  'dominant_emotion': 'fear'},
 'instance_2': {'emotion': {'angry': 1.4425331726670265,
   'disgust': 3.4158283312990534e-06,
   'fear': 25.449654459953308,
   'happy': 0.0001258898123523977,
   'sad': 41.07617735862732,
   'surprise': 0.0029570655897259712,
   'neutral': 32.028546929359436},
  'dominant_emotion': 'sad'},
 'instance_3': {'emotion': {'angry': 5.500451152329333e-05,
   'disgust': 7.720449713560784e-11,
   'fear': 23.536570370197296,
   'happy': 0.7447648793458939,
   'sad': 0.7199554238468409,
   'surprise': 0.06863058661110699,
   'neutral': 74.93002414703369},
  'dominant_emotion': 'neutral'},
 'instance_4': {'emotion': {'angry': 0.07466620574027827,
   'disgust': 2.908545626786653e-09,
   'fear': 0.26197283665

# Inserting the data into a Pandas dataframe so we can manipulate it

In [9]:
import pandas as pd

data = df_test2

emotions = set()
# First we need to find out all unique emotions
for key, value in data.items():
    for emotion in value['emotion'].keys():
        emotions.add(emotion)

rows = []
columns = ['instance'] + list(emotions)

for key, value in data.items():
    rows.append([0] * len(columns)) # Start creating a new row with zeros

    key = key.split('_')[1] # Get the 1,2,3 out of the instance
    rows[-1][0] = key
    for emotion, emotion_value in value['emotion'].items():
        rows[-1][columns.index(emotion)] = emotion_value # place the emotion in the correct index


df = pd.DataFrame(rows, columns=columns)
df.set_index('instance', inplace=True)

print(df)

               neutral       disgust      surprise     happy       fear  \
instance                                                                  
1         6.758833e+00  3.069457e-05  1.260055e-04  0.000084  47.920957   
2         3.202855e+01  3.415828e-06  2.957066e-03  0.000126  25.449654   
3         7.493002e+01  7.720450e-11  6.863059e-02  0.744765  23.536570   
4         9.580631e+01  2.908546e-09  5.318682e-06  0.000156   0.261973   
5         1.043748e+01  2.204510e-05  5.427929e-05  0.000046  22.268528   
...                ...           ...           ...       ...        ...   
1859      8.178790e+01  1.543939e-06  1.661720e-03  0.000014   1.078201   
1860      3.353570e+00  8.670942e-21  5.102676e-06  0.159173  96.473002   
1861      5.268770e+01  4.377116e-05  5.651602e-04  0.000009   4.592633   
1862      6.891441e+01  3.439967e-06  1.154030e-03  0.000164   7.345615   
1863      6.809531e-08  1.171775e-12  3.606429e-12  0.000208   0.157250   

                 angry  

# Saving it to a csv file so ur mum can open it in excel
* Using this weird export package Otherwise Jupyter starts being a knob

In [10]:
import tkinter as tk
from tkinter import filedialog

In [11]:
export_file_path = r'C:/Users/chazzers/Desktop/Recordings/Results'

In [ ]:
root= tk.Tk()

canvas1 = tk.Canvas()
canvas1.pack()

def exportCSV ():
    global df
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
    df.to_csv (export_file_path, index = True, header=True)

saveAsButton_CSV = tk.Button(text='Export CSV', command=exportCSV, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=saveAsButton_CSV)

root.mainloop()

# Plot it boiii
* Et voila! Dinner is served. nerd.

In [15]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
#test plot below